## Applying Regression techniques to Birch and Swinnerton-Dyer conjecture
We have the following conjecture about elipptic curves: $special\_value \cdot torsion^2 = sha \cdot real\_period \cdot regulator \cdot tamagawa\_product$

can we use regression techniques in computer science to model this problem?

- Verification with Linear Regression
- discovering new relations with Polynomial Regression

To prepare for this notebook, download the BSD dataset at https://math.mit.edu/~drew/mldata/bsd.txt and put it to the same director as this notebook


### Verifying the relations with Linear Regression

In [55]:
import numpy as np
from sklearn.linear_model import LinearRegression
import time
from sklearn.model_selection import train_test_split
import math

In [1]:
with open('./bsd.txt') as f:
    data = [l for l in f.readlines()]
data = [item.split(':') for item in data]

##### Spliting data to inputs (sha, real period, regulator and tamagawa product) and outputs (special value * torsion ^2)

- To apply linear regression, we need to take the Log of both side of the equation, so we have:
    - $\log(special\_value \cdot torsion^2) = \log(sha) + \log(real\_period) + \log(regulator) + \log(tamagawa\_product)$

In [76]:
targets = []
inputs = []
log_targets = []
log_inputs = []
for point in data:
    special_value, torsion = float(point[1]), int(point[2])
    sha, real_period, regulator, tagamawa_product = int(point[3]), float(point[4]), float(point[5]), int(point[6])
    output = special_value * (torsion**2)
    targets.append(output)
    inputs.append([sha, real_period, regulator, tagamawa_product]) 
    
    log_output = math.log(output)
    log_targets.append(log_output)
    log_inputs.append([math.log(sha), math.log(real_period), math.log(regulator), math.log(tagamawa_product)])

In [77]:
X_train, X_test, y_train, y_test = train_test_split(log_inputs, log_targets, test_size=0.3)

In [111]:
model = LinearRegression()

In [112]:
start = time.time_ns()/1000000
model.fit(X_train,y_train)
end = time.time_ns()/1000000
print('Linear regression takes ', str(end-start), 'milliseconds')

Linear regression takes  4932.36962890625 milliseconds


In [113]:
y_pred = model.predict(X_test)


In [114]:
model.score(X_test,y_test)

1.0

In [117]:
print(model.intercept_)
print(model.coef_)

-3.196642950342721e-11
[1. 1. 1. 1.]


the intercept is almost $0$ or negligible, and all the coefficient of the inputs are $1$, thus confirm the speculation through these examples.

In [118]:
#  Showing the model prediction: true output ~ predicted_output =log(sha) + log(real_period) + log(regulator) + log(tamagawa_product)
test_input = log_inputs[100]
test_output = log_targets[100]
print(model.intercept_ + np.dot(model.coef_, test_input), test_output)


3.2662017142608697 3.266201714292836


#### Showcasing the predicted results

In [84]:
import pandas as pd
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df


,Actual,Predicted
0,1.702233,1.702233
1,0.204636,0.204636
2,3.569592,3.569592
3,0.495843,0.495843
4,3.248798,3.248798
5,2.995703,2.995703
6,3.399571,3.399571
7,3.444722,3.444722
8,2.043000,2.043000
9,2.441504,2.441504


### Approximate BSD with Polynomial Regression

we can also approximate $special\_value \cdot torsion^2$ with the 4 input values using the polynomial Regression techniques.
In this example, we operate directly on the input values, not the $\ln$ of the inputs

In [85]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(inputs, targets, test_size=0.3)

In [88]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

degree=3
polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg.fit(X_train_orig,y_train_orig)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('linearregression', LinearRegression())])

In [89]:
y_pred_poly = polyreg.predict(X_test_orig)
y_train_pred_poly = polyreg.predict(X_train_orig)

In [94]:
df = pd.DataFrame({'Actual': y_train_orig, 'Predicted': y_train_pred_poly})
df

,Actual,Predicted
0,3.226683,3.240413
1,89.254600,89.213342
2,11.894917,11.870497
3,4.185799,4.297215
4,19.763617,19.750329
5,5.282492,5.282106
6,38.637670,38.621750
7,2.556541,2.587699
8,8.981322,9.014454
9,68.089808,69.534225


In [92]:
polyreg = LinearRegression()
polynomial_features= PolynomialFeatures(degree=3)
x_poly = polynomial_features.fit_transform(X_train_orig)

In [93]:
model.fit(x_poly, y_train_orig)
y_poly_pred = model.predict(x_poly)

In [95]:
df = pd.DataFrame({'Actual': y_train_orig, 'Predicted': y_poly_pred})
df


,Actual,Predicted
0,3.226683,3.240413
1,89.254600,89.213342
2,11.894917,11.870497
3,4.185799,4.297215
4,19.763617,19.750329
5,5.282492,5.282106
6,38.637670,38.621750
7,2.556541,2.587699
8,8.981322,9.014454
9,68.089808,69.534225


In [105]:
x_poly[0,:].shape

(35,)

In [109]:
np.dot(model.coef_, x_poly[0,:]) + model.intercept_

3.2404128911177144

##### Output of Polynomial Regression

from the above output, PolynomialFeatures predicted some polynomial coefficients for all polynomial combinations of the features with degree less than or equal to the specified degree (in this case 3rd degree). In other words, for 4 variables, we have a polynomial consists of : $1, a, b,c,d, a^2, ab, ac , ad, b^2 \cdots a^3, a^2b, \cdots a^4 \cdots$. There are:
- $C(6, 3)$ terms for $(a+b+c+d)^4$
- $C(5,2)$ terms for $(a+b+c+d)^3$
- $C(4,1)$ terms for $(a+b+c+d)^2$
- $C(3, 0)$ terms for $(a+b+c+d)^1$
- thus we have a polynomial with 35 terms